In [2]:
import numpy as np 
import pandas as pd
from pandas.plotting import lag_plot
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.api as sm
TICKER = "SPY"

INTERVAL = "1d"
if INTERVAL == "1h":
    PERIOD = "730d"
else: 
    PERIOD = "max"

SHIFT = 1 
LOOKBACK = 10000

STRATEGY = ["Volume"]

def get_data(ticker = TICKER, interval = INTERVAL, lookback = LOOKBACK):

    df = yf.download(ticker, interval = interval, period = PERIOD)
    df.columns = df.columns.get_level_values(0)
    df = df.reset_index(drop=True)

    return df.iloc[-lookback:, :]

# Add Features
# Add Target 
def add_target(df, shift = SHIFT):

    df[f"Close + {shift}"] = df["Close"].shift(-shift)
    df["Target"] = df[f"Close + {shift}"] > df["Close"]

    return df

# --- (5) Generate Regression Output ---
def generate_regression_output(df, features=STRATEGY, target="Target"):

    subset = df[features + [target]].dropna()
    print(subset.columns)
    X = subset[features] 
    y = subset[target] 
    X_with_const = sm.add_constant(X)
    
    model = sm.Logit(y, X_with_const).fit() 
    print(model.summary())

    return df


# Validate Regression Output

def main():

    df = get_data()
    df = add_target(df)
    df = generate_regression_output(df)

    return df 

df = main()
df


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['SPY']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Index(['Volume', 'Target'], dtype='object', name='Price')


ValueError: zero-size array to reduction operation maximum which has no identity